# Tool Ahead of Time (TAoT) Tutorial

Lets jump straight into this tutorial, as time waits for no one 😊

This tutorial uses the DeepSeek-R1-0528 685B model, but this tutorial can also be applied to other models available through Langchain's ChatOpenAI class.

First, pip install the taot package, as below:

In [ ]:
%pip install taot

If you haven't already, also pip install the other dependencies required in this tutorial:

In [ ]:
%pip install langchain-core langchain-openai

## Creating Tools

Next, we create tool functions using LangChain's `@tool` decorator.

This is just any function (with inputs and outputs) and `@tool` added at the top of the function.

I have created two tool functions 'calculator' and 'text_analyzer' below:

In [1]:
from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    """Evaluate a math expression."""
    try:
        expression = expression.strip()
        if not expression:
            return "Error: Empty expression"
        
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
            
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def text_analyzer(text: str, analysis_type: str) -> str:
    """
    Analyze text to count either words or characters.
    
    Args:
        text (str): The text to analyze
        analysis_type (str): Either 'words' or 'chars'
    """
    try:
        text = text.strip()
        if not text:
            return "Error: Empty text"
            
        if analysis_type.lower() == 'words':
            word_count = len(text.split())
            return f"{word_count}"
        elif analysis_type.lower() == 'chars':
            char_count = len(text)
            return f"{char_count}"
        else:
            return "Error: analysis_type must be either 'words' or 'chars'"
    except Exception as e:
        return f"Error: {str(e)}"

## Initialize Model

Now, initialize a model instance using the format below. 

In this tutorial, I am using the DeepSeek-R1-0528 685B model hosted on the platform OpenRouter. This model hosted on OpenRouter is available on Langchain's ChatOpenAI class.

If you want to use another model, you will need to check if your model (hosted on whichever platform you have chosen, for eg. Azure, Together AI or DeepSeek's own platform etc.) is first available on Langchain's ChatOpenAI class, and then change the values of the parameters `model`, `api_key` and `base_url` below according to which model and platform you have chosen.

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os

# Load environment variable (ie. API key) from .env file
load_dotenv()

# Initialize model
model = ChatOpenAI(
    model="deepseek/deepseek-r1-0528",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1"
)

## Previous Messages

Next, if you already have a history of previous messages between the user and the chatbot, store them in the format below.

Note: The format of the previous messages does not include the system message (which we will define later further down in this notebook). This design is chosen according to current best practices in chatbot design where we isolate the system message from previous messages.

In [3]:
# Example previous messages
previous_messages = [
    # {"role": "system", "content": "You are a helpful AI assistant."}, # Commented out as we do not include system message
    {"role": "user", "content": "What is the capital of Australia?"},
    {"role": "assistant", "content": "The capital of Australia is Canberra."}
]

## Getting Model Response

Finally, now the fun part where we get to see the response of the model using tool calling! 🛠️

For ease of use, I have designed the taot package to mimic LangChain's and LangGraph's `create_react_agent` method with tool calling, ie. the taot package follows a similar method to LangChain's and LangGraph's:

```
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools=[])
response = agent_executor.invoke({"messages": all_messages})
print(response["messages"][-1].content)
```

First, the `system_message` variable below can start with any customized system message as per usual, for eg. "You are a helpful assistant. ", "You are an expert programmer in Python. ", "You are a world class expert in SEO optimization. " etc.

Then, the `system_message` variable below needs to **STRICTLY** include the following: "You are an assistant with access to specific tools. When the user's question requires a {tool use}, use the {'corresponding'} tool. For the {'corresponding'} tool, provide the {user message} as a string into the {'user message'} argument in the tool or any {'predefined values'} as a string for other arguments in the tool."

For eg. for the 'calculator' tool, since the function for the 'calculator' tool above has one argument called 'expression', the `system_message` variable below would need to look like "You are a math expert. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool."

For the 'text_analyze' tool, since the function for the 'text_analyze' tool above has two arguments 'text' and 'analysis_type' (where the 'analysis_type' argument has two predefined values 'words' and 'chars'), the `system_message` variable below would need to look like "You are an expert in linguitics. You are an assistant with access to specific tools. When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."

Below are five examples of different combinations of user questions and tools used:

In [4]:
from taot import create_system_message_taot, create_react_agent_taot

# Example for calculator tool only
system_message = "You are a math expert. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool."
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "What is 123 * 456?"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[calculator])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])

# Example for text analyzer tool only
system_message = "You are an expert in linguitics. You are an assistant with access to specific tools. When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "How many words are in this sentence?: I built my 1st Hello World program"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[text_analyzer])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])

# Example for both tools with user question requiring math calculation
system_message = """You are an expert in math and linguitics. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool.
When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."""
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "What is 123 * 456?"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[calculator, text_analyzer])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])

# Example for both tools with user question requiring analysis the text
system_message = """You are an expert in math and linguitics. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool.
When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."""
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "How many words are in this sentence?: I built my 1st Hello World program"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[calculator, text_analyzer])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])

# Example for both tools with user question not requiring any tools
system_message = """You are an expert in math and linguitics. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool.
When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."""
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "How many languages are there in the world?"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[calculator, text_analyzer])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])


The product of 123 and 456 is 56,088.
The sentence "I built my 1st Hello World program" contains **7 words**.
The product of 123 and 456 is 56,088.
The sentence "I built my 1st Hello World program" contains 7 words.
It's difficult to provide an exact number because the definition of what constitutes a distinct language versus a dialect is subjective. However, according to Ethnologue (the most comprehensive catalog of world languages), there are approximately 7,168 living languages as of 2024. This number constantly changes as languages evolve or become extinct.


## Summary

Putting all the scripts above together:

In [5]:
from langchain_core.tools import tool
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from taot import create_system_message_taot, create_react_agent_taot

@tool
def calculator(expression: str) -> str:
    """Evaluate a math expression."""
    try:
        expression = expression.strip()
        if not expression:
            return "Error: Empty expression"
        
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
            
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def text_analyzer(text: str, analysis_type: str) -> str:
    """
    Analyze text to count either words or characters.
    
    Args:
        text (str): The text to analyze
        analysis_type (str): Either 'words' or 'chars'
    """
    try:
        text = text.strip()
        if not text:
            return "Error: Empty text"
            
        if analysis_type.lower() == 'words':
            word_count = len(text.split())
            return f"{word_count}"
        elif analysis_type.lower() == 'chars':
            char_count = len(text)
            return f"{char_count}"
        else:
            return "Error: analysis_type must be either 'words' or 'chars'"
    except Exception as e:
        return f"Error: {str(e)}"
    
# Load environment variable (ie. API key) from .env file
load_dotenv()

# Initialize model
model = ChatOpenAI(
    model="deepseek/deepseek-r1-0528",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1"
)

# Example previous messages
previous_messages = [
    # {"role": "system", "content": "You are a helpful AI assistant."}, # Commented out as we do not include system message
    {"role": "user", "content": "What is the capital of Australia?"},
    {"role": "assistant", "content": "The capital of Australia is Canberra."}
]

# Example for calculator tool only
system_message = "You are a math expert. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool."
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "What is 123 * 456?"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[calculator])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])

# Example for text analyzer tool only
system_message = "You are an expert in linguitics. You are an assistant with access to specific tools. When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "How many words are in this sentence?: I built my 1st Hello World program"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[text_analyzer])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])

# Example for both tools with user question requiring math calculation
system_message = """You are an expert in math and linguitics. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool.
When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."""
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "What is 123 * 456?"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[calculator, text_analyzer])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])

# Example for both tools with user question requiring analysis the text
system_message = """You are an expert in math and linguitics. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool.
When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."""
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "How many words are in this sentence?: I built my 1st Hello World program"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[calculator, text_analyzer])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])

# Example for both tools with user question not requiring any tools
system_message = """You are an expert in math and linguitics. You are an assistant with access to specific tools. When the user's question requires a calculation, use the 'calculator' tool. For the 'calculator' tool, provide the user provided math expression as a string into the 'expression' argument in the tool.
When the user's question requires analysis of the text provided by the user, use the 'text_analyzer' tool. For the 'text_analyzer' tool, provide the user provided text as a string into the 'text' argument in the tool and either 'words' or 'chars' as a string into the 'analysis_type' argument in the tool."""
system_message_taot = create_system_message_taot(system_message)
all_messages = [{"role": "system", "content": system_message_taot}]
# Add previous messages (if available)
all_messages.extend(previous_messages)
# Add current user prompt
user_message = "How many languages are there in the world?"
all_messages.append({"role": "user", "content": user_message})
# Get model response
agent_executor_taot = create_react_agent_taot(model, tools=[calculator, text_analyzer])
response = agent_executor_taot.invoke({"messages": all_messages})
print(response['messages'][0]['content'])



The product of 123 and 456 is 56,088.
The sentence "I built my 1st Hello World program" contains **7 words**.
The product of 123 and 456 is 56,088.
The sentence "I built my 1st Hello World program" contains **7 words**.
Estimating the exact number of languages in the world is challenging due to factors like dialects, endangered languages, and regional variations. However, linguistic databases such as _Ethnologue_ currently document approximately **7,100–7,400 living languages** worldwide. This number fluctuates as languages evolve or become extinct.
